# Solution - 分月预测

### 导入相关库

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' 

np.random.seed(2019)

### 读取数据

In [2]:
data_path = './data/' 
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

train_sales_data = train_sales_data.merge(train_search_data, on=['province', 'adcode', 'model', 'regYear', 'regMonth'])

### 固定车型和省份顺序-样本顺序

In [3]:
cars = ['f8a6975573af1b33', '2a2ab41f8f6ff1cb', 'd4efbebb087fd03f', '3e21824be728cbec', 'ea489c253676aafc', '6155b214590c66e6', 'fc32b1a017b34efe', '9c1c7ee8ebdda299', 'fde95ea242abd896', '7a7885e2d7c00bcf', '7245e0ee27b195cd', 'b25c4e2e3856af22', '7aab7fca2470987e', 'feabbf46658382b9', '04e66e578f653ab9', '5d7fb682edd0f937', 'b4be3a4917289c82', '54fc07138d70374c', 'ef76a85c4b39f693', 'bb9fbec9a2833839', '3c974920a76ac9c1', '212083a9246d2fd3', '4f79773e600518a6', 'af6f4f548684e14d', '936168bd4850913d', 'cd5841d44fd7625e', '0797526c057dcf5b', 'a207df29ec9583f0', '3d7554f1f56dd664', '7023efdab9cedc03', 'da457d15788fe8ee', '12f8b7e14947c34d', '28e29f2c03dcd84c', '63065128401bb3ff', 'a432c483b5beb856', '37aa9169b575ef79', '17bc272c93f19d56', '61e73e32ad101892', '4a103c30d593fbbe', '2d0d2c3403909fdb', '6858d6dfe680bdf7', '17363f08d683d52b', '346393c2c6305fb1', '5b1c11c3efed5312', '97f15de12cfabbd5', 'a9a43d1a7ecbe75d', '7cf283430b3b5e38', 'c6833cb891626c17', 'a28bb927b6fcb33c', 'dff803b4024d261d', '02aab221aabc03b9', 'f5d69960089c3614', '06880909932890ca', '79de4e4b24c35b04', 'd0f245b8781e3631', 'c06a2a387c0ee510', 'cc21c7e91a3b5a0c', 'f270f6a489c6a9d7', '8c915fe4632fb9fa', 'c6cd4e0e073f5ac2']
provinces = ['浙江', '福建', '四川', '陕西', '安徽', '湖南', '广东', '云南', '上海', '山东', '湖北', '黑龙江', '江苏', '广西', '内蒙古', '辽宁', '北京', '重庆', '河北', '山西', '江西', '河南']

### 评测函数

In [4]:
from sklearn.metrics import mean_squared_error

def metrics(y_true, y_pred, model):
    data = pd.DataFrame({'model': model, 'salesVolume': y_true, 'label': y_pred})
    data['label'] = data['label'].map(lambda index: -index if index < 0 else index)
    res, count = 0, 0
    for index, cars in data.groupby('model'):
        a = np.array(cars['salesVolume'])
        b = np.array(cars['label'])
        temp = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
        res += temp
        count += 1
        print(temp)
    return 1 - (res / count)

### 获取训练/测试数据索引下标

In [5]:
def get_train_feature(windows_size, before):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
            car_province_part['label'] = car_province_part['salesVolume'].shift(-windows_size)
            car_province_part = car_province_part[before:24 - windows_size]
            features = pd.concat([features, car_province_part], axis=0)
    features.index = range(len(features))
    return features

def get_test_feature(windows_size, before):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
            car_province_part['label'] = car_province_part['salesVolume'].shift(-windows_size)
            car_province_part = car_province_part[-1:]
            features = pd.concat([features, car_province_part], axis=0)
    features.index = range(len(features))
    return features

### 特征提取

In [6]:
def get_basic_feature(windows_size, before, data_set_name):
    features = pd.DataFrame()
    for car in cars:
        for province in provinces:
            car_province_part = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)].copy()
            car_province_part['popularity'] = car_province_part['popularity'].apply(lambda index: np.log(index))     ###
            car_province_part['salesVolume'] = car_province_part['salesVolume'].apply(lambda index: np.log(index))   ###
            
            # 春节标记特征
            car_province_part['is_pring_festival'] = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            car_province_part['distance_spring_festival'] = [1, 0, 1, 2, 3, 4, 5, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 6, 5, 4, 3, 2]
            
            # 一阶差分
            for index in range(1, before, 1):
                car_province_part['salesVolume_' + str(index)] = car_province_part['salesVolume'].shift(index)
                car_province_part['salesVolume_diff_' + str(index)] = car_province_part['salesVolume'].diff(index)
                car_province_part['salesVolume_qoq_' + str(index)] = car_province_part['salesVolume'] / car_province_part['salesVolume_' + str(index)]

                car_province_part['popularity_' + str(index)] = car_province_part['popularity'].shift(index)
                car_province_part['popularity_diff_' + str(index)] = car_province_part['popularity'].diff(index)
                car_province_part['popularity_hb_' + str(index)] = car_province_part['popularity'] / car_province_part['popularity_' + str(index)]

            # 二阶差分
            for index in range(1, before - 1, 1):   
                car_province_part['salesVolume_diff2_{}'.format(str(index))] = car_province_part['salesVolume_diff_' + str(index)].diff(1)

            # 历史统计特征
            salesVolume = list(car_province_part['salesVolume'])
            popularity = list(car_province_part['popularity'])
            car_province_part['index'] = 1
            car_province_part['index'] = car_province_part['index'].cumsum()
            car_province_part['salesVolume_his'] = car_province_part['index'].map(lambda index: salesVolume[index - 7:index])
            car_province_part['popularity_his'] = car_province_part['index'].map(lambda index: popularity[index - 7:index])

            car_province_part['salesVolume_his_diff'] = car_province_part['salesVolume_his'].map(lambda index: np.diff(index))
            car_province_part['popularity_his_diff'] = car_province_part['popularity_his'].map(lambda index: np.diff(index))

            def pth(array):
                return np.max(array) - np.min(array)

            fea_name = ['max', 'min', 'aver', 'var', 'pth']
            fun_name = [np.max, np.min, np.average, np.var, pth]
            for i in range(len(fun_name)):
                car_province_part['salesVolume_his_' + fea_name[i]] = car_province_part['salesVolume_his'].apply(lambda index: 0 if len(index) == 0 else fun_name[i](index))                
                car_province_part['salesVolume_his_diff_' + fea_name[i]] = car_province_part['salesVolume_his_diff'].apply(lambda index: 0 if len(index) == 0 else fun_name[i](index))
            
            car_province_part.drop(['index', 'salesVolume_his', 'popularity_his', 'salesVolume_his_diff', 'popularity_his_diff'], axis=1, inplace=True)
            
            # 数据集划分
            if data_set_name == 'train':
                car_province_part = car_province_part[before : 24 - windows_size]
            else:
                car_province_part = car_province_part[-1:]

            car_province_part.drop(['popularity'], axis=1, inplace=True)    ###  , 'day_count', 'day_salesVolume', 'popularity'
            features = pd.concat([features, car_province_part], axis=0, ignore_index=True)

    print(features)
    return features

### Begain

In [7]:
test_prob_collection = pd.DataFrame()

### Model-LightGBM - 一月

In [8]:
size, pre = 1, 10  # 4
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

categorial_name = [0, 1, 2, 3, 4, 6, 7]
drop_cols = ['salesVolume', 'popularity']

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 1
###############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 11)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

      province  adcode             model bodyType  regYear  regMonth  \
0           浙江  330000  f8a6975573af1b33    Sedan     2016        11   
1           浙江  330000  f8a6975573af1b33    Sedan     2016        12   
2           浙江  330000  f8a6975573af1b33    Sedan     2017         1   
3           浙江  330000  f8a6975573af1b33    Sedan     2017         2   
4           浙江  330000  f8a6975573af1b33    Sedan     2017         3   
...        ...     ...               ...      ...      ...       ...   
17155       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         7   
17156       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         8   
17157       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         9   
17158       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        10   
17159       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        11   

       salesVolume  is_pring_festival  distance_spring_festival  \
0         6.543912                  0                         2   
1

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2016,11,1454.0,6.543912,0,2,...,6.752270,0.190701,6.419995,-0.208359,6.593033,-0.016429,0.012979,0.020524,0.332275,0.399060
1,浙江,330000,f8a6975573af1b33,Sedan,2016,12,752.0,7.282074,0,1,...,7.282074,0.738162,6.419995,-0.208359,6.684402,0.135729,0.072106,0.088094,0.862079,0.946520
2,浙江,330000,f8a6975573af1b33,Sedan,2017,1,503.0,6.622736,1,0,...,7.282074,0.738162,6.419995,-0.659337,6.706551,0.033790,0.065451,0.177449,0.862079,1.397499
3,浙江,330000,f8a6975573af1b33,Sedan,2017,2,780.0,6.220590,0,1,...,7.282074,0.738162,6.220590,-0.659337,6.678064,-0.065018,0.086645,0.195256,1.061483,1.397499
4,浙江,330000,f8a6975573af1b33,Sedan,2017,3,679.0,6.659294,0,2,...,7.282074,0.738162,6.220590,-0.659337,6.685007,-0.009146,0.085999,0.229692,1.061483,1.397499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17155,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,7,991.0,6.577861,0,6,...,6.899723,0.470004,5.768321,-1.131402,6.411382,-0.053644,0.107535,0.273930,1.131402,1.601406
17156,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,8,1189.0,6.898715,0,6,...,6.898715,0.470004,5.768321,-0.169675,6.411238,0.188399,0.107394,0.045126,1.130394,0.639678
17157,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,9,1206.0,7.080868,0,5,...,7.080868,0.320853,6.238325,-0.169675,6.598745,0.140424,0.077244,0.029614,0.842543,0.490528
17158,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,10,1267.0,7.095064,0,4,...,7.095064,0.320853,6.322565,-0.169675,6.721136,0.100471,0.078898,0.028527,0.772499,0.490528


     province  adcode             model bodyType  regYear  regMonth  \
0          浙江  330000  f8a6975573af1b33    Sedan     2017        12   
1          福建  350000  f8a6975573af1b33    Sedan     2017        12   
2          四川  510000  f8a6975573af1b33    Sedan     2017        12   
3          陕西  610000  f8a6975573af1b33    Sedan     2017        12   
4          安徽  340000  f8a6975573af1b33    Sedan     2017        12   
...       ...     ...               ...      ...      ...       ...   
1315       重庆  500000  c6cd4e0e073f5ac2    Sedan     2017        12   
1316       河北  130000  c6cd4e0e073f5ac2    Sedan     2017        12   
1317       山西  140000  c6cd4e0e073f5ac2    Sedan     2017        12   
1318       江西  360000  c6cd4e0e073f5ac2    Sedan     2017        12   
1319       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        12   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        7.357556                  0                         2    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
                            ..
salesVolume_his_diff_aver    0
salesVolume_his_var          0
salesVolume_his_diff_var     0
salesVolume_his_pth          0
salesVolume_his_diff_pth     0
Length: 82, dtype: int64

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


In [9]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,    # 2 ** 5 - 1
      # 'min_child_samples': 100,
      'max_depth': 6,    # 6
      'subsample': 0.8,   # 0.8
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2020,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# num_boost_round: 5000   early_stopping_rounds:100
module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

# feature importance
importance = module.feature_importance()
print(importance)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)   
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)
print(train_feature.shape)

[1]	valid_0's rmse: 1.67504
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.63237
[3]	valid_0's rmse: 1.59086
[4]	valid_0's rmse: 1.54992
[5]	valid_0's rmse: 1.51207
[6]	valid_0's rmse: 1.47497
[7]	valid_0's rmse: 1.43847
[8]	valid_0's rmse: 1.40274
[9]	valid_0's rmse: 1.36713
[10]	valid_0's rmse: 1.3351
[11]	valid_0's rmse: 1.30436
[12]	valid_0's rmse: 1.2757
[13]	valid_0's rmse: 1.24561
[14]	valid_0's rmse: 1.21674
[15]	valid_0's rmse: 1.18851
[16]	valid_0's rmse: 1.16289
[17]	valid_0's rmse: 1.13849
[18]	valid_0's rmse: 1.11441
[19]	valid_0's rmse: 1.08889
[20]	valid_0's rmse: 1.06536
[21]	valid_0's rmse: 1.04338
[22]	valid_0's rmse: 1.02227
[23]	valid_0's rmse: 1.0017
[24]	valid_0's rmse: 0.982387
[25]	valid_0's rmse: 0.964213
[26]	valid_0's rmse: 0.946937
[27]	valid_0's rmse: 0.928912
[28]	valid_0's rmse: 0.912121
[29]	valid_0's rmse: 0.895491
[30]	valid_0's rmse: 0.878735
[31]	valid_0's rmse: 0.863213
[32]	valid_0's rmse: 0.848952
[33]	valid_0

[285]	valid_0's rmse: 0.467548
[286]	valid_0's rmse: 0.467546
[287]	valid_0's rmse: 0.467577
[288]	valid_0's rmse: 0.467618
[289]	valid_0's rmse: 0.467655
[290]	valid_0's rmse: 0.467454
[291]	valid_0's rmse: 0.467443
[292]	valid_0's rmse: 0.467248
[293]	valid_0's rmse: 0.467369
[294]	valid_0's rmse: 0.467408
[295]	valid_0's rmse: 0.467167
[296]	valid_0's rmse: 0.467135
[297]	valid_0's rmse: 0.467285
[298]	valid_0's rmse: 0.467351
[299]	valid_0's rmse: 0.467381
[300]	valid_0's rmse: 0.467237
[301]	valid_0's rmse: 0.467188
[302]	valid_0's rmse: 0.467252
[303]	valid_0's rmse: 0.467469
[304]	valid_0's rmse: 0.467268
[305]	valid_0's rmse: 0.467316
[306]	valid_0's rmse: 0.467023
[307]	valid_0's rmse: 0.466938
[308]	valid_0's rmse: 0.466955
[309]	valid_0's rmse: 0.466956
[310]	valid_0's rmse: 0.466794
[311]	valid_0's rmse: 0.466845
[312]	valid_0's rmse: 0.467118
[313]	valid_0's rmse: 0.467085
[314]	valid_0's rmse: 0.467135
[315]	valid_0's rmse: 0.467098
[316]	valid_0's rmse: 0.467028
[317]	va

[564]	valid_0's rmse: 0.458179
[565]	valid_0's rmse: 0.458178
[566]	valid_0's rmse: 0.458112
[567]	valid_0's rmse: 0.45805
[568]	valid_0's rmse: 0.458019
[569]	valid_0's rmse: 0.457924
[570]	valid_0's rmse: 0.457926
[571]	valid_0's rmse: 0.457933
[572]	valid_0's rmse: 0.45793
[573]	valid_0's rmse: 0.458055
[574]	valid_0's rmse: 0.458077
[575]	valid_0's rmse: 0.457919
[576]	valid_0's rmse: 0.457899
[577]	valid_0's rmse: 0.45765
[578]	valid_0's rmse: 0.457551
[579]	valid_0's rmse: 0.45743
[580]	valid_0's rmse: 0.457596
[581]	valid_0's rmse: 0.457622
[582]	valid_0's rmse: 0.457581
[583]	valid_0's rmse: 0.457494
[584]	valid_0's rmse: 0.45765
[585]	valid_0's rmse: 0.457605
[586]	valid_0's rmse: 0.457387
[587]	valid_0's rmse: 0.4575
[588]	valid_0's rmse: 0.457564
[589]	valid_0's rmse: 0.45739
[590]	valid_0's rmse: 0.457455
[591]	valid_0's rmse: 0.457513
[592]	valid_0's rmse: 0.457717
[593]	valid_0's rmse: 0.457805
[594]	valid_0's rmse: 0.457892
[595]	valid_0's rmse: 0.458002
[596]	valid_0's 

```
model train over, rmse: 0.6850039724275112

```

### Model-LightGBM - 二月

In [10]:
size, pre = 2, 9  # 4
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 2
######################################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 10)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

      province  adcode             model bodyType  regYear  regMonth  \
0           浙江  330000  f8a6975573af1b33    Sedan     2016        10   
1           浙江  330000  f8a6975573af1b33    Sedan     2016        11   
2           浙江  330000  f8a6975573af1b33    Sedan     2016        12   
3           浙江  330000  f8a6975573af1b33    Sedan     2017         1   
4           浙江  330000  f8a6975573af1b33    Sedan     2017         2   
...        ...     ...               ...      ...      ...       ...   
17155       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         6   
17156       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         7   
17157       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         8   
17158       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         9   
17159       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        10   

       salesVolume  is_pring_festival  distance_spring_festival  \
0         6.752270                  0                         3   
1

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2016,10,1454.0,6.752270,0,3,...,6.752270,0.190701,6.419995,-0.174788,6.598675,0.028144,0.012616,0.013348,0.332275,0.365489
1,浙江,330000,f8a6975573af1b33,Sedan,2016,11,752.0,6.543912,0,2,...,6.752270,0.190701,6.419995,-0.208359,6.593033,-0.016429,0.012979,0.020524,0.332275,0.399060
2,浙江,330000,f8a6975573af1b33,Sedan,2016,12,503.0,7.282074,0,1,...,7.282074,0.738162,6.419995,-0.208359,6.684402,0.135729,0.072106,0.088094,0.862079,0.946520
3,浙江,330000,f8a6975573af1b33,Sedan,2017,1,780.0,6.622736,1,0,...,7.282074,0.738162,6.419995,-0.659337,6.706551,0.033790,0.065451,0.177449,0.862079,1.397499
4,浙江,330000,f8a6975573af1b33,Sedan,2017,2,679.0,6.220590,0,1,...,7.282074,0.738162,6.220590,-0.659337,6.678064,-0.065018,0.086645,0.195256,1.061483,1.397499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17155,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,6,991.0,6.580639,0,5,...,7.367709,0.470004,5.768321,-1.131402,6.524217,-0.131178,0.221495,0.296100,1.599388,1.601406
17156,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,7,1189.0,6.577861,0,6,...,6.899723,0.470004,5.768321,-1.131402,6.411382,-0.053644,0.107535,0.273930,1.131402,1.601406
17157,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,8,1206.0,6.898715,0,6,...,6.898715,0.470004,5.768321,-0.169675,6.411238,0.188399,0.107394,0.045126,1.130394,0.639678
17158,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,9,1267.0,7.080868,0,5,...,7.080868,0.320853,6.238325,-0.169675,6.598745,0.140424,0.077244,0.029614,0.842543,0.490528


     province  adcode             model bodyType  regYear  regMonth  \
0          浙江  330000  f8a6975573af1b33    Sedan     2017        12   
1          福建  350000  f8a6975573af1b33    Sedan     2017        12   
2          四川  510000  f8a6975573af1b33    Sedan     2017        12   
3          陕西  610000  f8a6975573af1b33    Sedan     2017        12   
4          安徽  340000  f8a6975573af1b33    Sedan     2017        12   
...       ...     ...               ...      ...      ...       ...   
1315       重庆  500000  c6cd4e0e073f5ac2    Sedan     2017        12   
1316       河北  130000  c6cd4e0e073f5ac2    Sedan     2017        12   
1317       山西  140000  c6cd4e0e073f5ac2    Sedan     2017        12   
1318       江西  360000  c6cd4e0e073f5ac2    Sedan     2017        12   
1319       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        12   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        7.357556                  0                         2    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
                            ..
salesVolume_his_diff_aver    0
salesVolume_his_var          0
salesVolume_his_diff_var     0
salesVolume_his_pth          0
salesVolume_his_diff_pth     0
Length: 75, dtype: int64

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


In [11]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2020,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)   
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.67571
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.63366
[3]	valid_0's rmse: 1.59457
[4]	valid_0's rmse: 1.55521
[5]	valid_0's rmse: 1.51991
[6]	valid_0's rmse: 1.48352
[7]	valid_0's rmse: 1.44887
[8]	valid_0's rmse: 1.41323
[9]	valid_0's rmse: 1.38026
[10]	valid_0's rmse: 1.34871
[11]	valid_0's rmse: 1.32254
[12]	valid_0's rmse: 1.3003
[13]	valid_0's rmse: 1.27334
[14]	valid_0's rmse: 1.24702
[15]	valid_0's rmse: 1.22195
[16]	valid_0's rmse: 1.19702
[17]	valid_0's rmse: 1.17304
[18]	valid_0's rmse: 1.14936
[19]	valid_0's rmse: 1.12531
[20]	valid_0's rmse: 1.10409
[21]	valid_0's rmse: 1.08472
[22]	valid_0's rmse: 1.06466
[23]	valid_0's rmse: 1.04334
[24]	valid_0's rmse: 1.02436
[25]	valid_0's rmse: 1.00631
[26]	valid_0's rmse: 0.990637
[27]	valid_0's rmse: 0.973953
[28]	valid_0's rmse: 0.959098
[29]	valid_0's rmse: 0.943378
[30]	valid_0's rmse: 0.928062
[31]	valid_0's rmse: 0.916745
[32]	valid_0's rmse: 0.906983
[33]	valid_0

[277]	valid_0's rmse: 0.557302
[278]	valid_0's rmse: 0.557018
[279]	valid_0's rmse: 0.556906
[280]	valid_0's rmse: 0.557009
[281]	valid_0's rmse: 0.556993
[282]	valid_0's rmse: 0.556968
[283]	valid_0's rmse: 0.556882
[284]	valid_0's rmse: 0.556739
[285]	valid_0's rmse: 0.556475
[286]	valid_0's rmse: 0.556319
[287]	valid_0's rmse: 0.556027
[288]	valid_0's rmse: 0.555921
[289]	valid_0's rmse: 0.556083
[290]	valid_0's rmse: 0.555972
[291]	valid_0's rmse: 0.555567
[292]	valid_0's rmse: 0.555343
[293]	valid_0's rmse: 0.555339
[294]	valid_0's rmse: 0.555178
[295]	valid_0's rmse: 0.555177
[296]	valid_0's rmse: 0.555244
[297]	valid_0's rmse: 0.554971
[298]	valid_0's rmse: 0.555345
[299]	valid_0's rmse: 0.55526
[300]	valid_0's rmse: 0.555046
[301]	valid_0's rmse: 0.555073
[302]	valid_0's rmse: 0.555125
[303]	valid_0's rmse: 0.555086
[304]	valid_0's rmse: 0.554979
[305]	valid_0's rmse: 0.554959
[306]	valid_0's rmse: 0.554853
[307]	valid_0's rmse: 0.554725
[308]	valid_0's rmse: 0.554905
[309]	val

[543]	valid_0's rmse: 0.551439
[544]	valid_0's rmse: 0.551462
[545]	valid_0's rmse: 0.551433
[546]	valid_0's rmse: 0.551816
[547]	valid_0's rmse: 0.552037
[548]	valid_0's rmse: 0.552152
[549]	valid_0's rmse: 0.55239
[550]	valid_0's rmse: 0.552805
[551]	valid_0's rmse: 0.552731
[552]	valid_0's rmse: 0.552817
[553]	valid_0's rmse: 0.552767
[554]	valid_0's rmse: 0.55274
[555]	valid_0's rmse: 0.552668
[556]	valid_0's rmse: 0.552616
[557]	valid_0's rmse: 0.552403
[558]	valid_0's rmse: 0.552407
[559]	valid_0's rmse: 0.552397
[560]	valid_0's rmse: 0.552389
[561]	valid_0's rmse: 0.552322
[562]	valid_0's rmse: 0.552137
[563]	valid_0's rmse: 0.552063
[564]	valid_0's rmse: 0.552051
[565]	valid_0's rmse: 0.552021
[566]	valid_0's rmse: 0.551818
[567]	valid_0's rmse: 0.551769
[568]	valid_0's rmse: 0.551957
[569]	valid_0's rmse: 0.551989
[570]	valid_0's rmse: 0.552241
[571]	valid_0's rmse: 0.552413
[572]	valid_0's rmse: 0.552368
[573]	valid_0's rmse: 0.552218
[574]	valid_0's rmse: 0.552171
[575]	vali

```
model train over, rmse: 0.653147613361799
```

### Model-LightGBM - 三月

In [12]:
size, pre = 3, 8   # 5
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 3
##############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 9)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

      province  adcode             model bodyType  regYear  regMonth  \
0           浙江  330000  f8a6975573af1b33    Sedan     2016         9   
1           浙江  330000  f8a6975573af1b33    Sedan     2016        10   
2           浙江  330000  f8a6975573af1b33    Sedan     2016        11   
3           浙江  330000  f8a6975573af1b33    Sedan     2016        12   
4           浙江  330000  f8a6975573af1b33    Sedan     2017         1   
...        ...     ...               ...      ...      ...       ...   
17155       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         5   
17156       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         6   
17157       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         7   
17158       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         8   
17159       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         9   

       salesVolume  is_pring_festival  distance_spring_festival  \
0         6.714171                  0                         4   
1

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2016,9,1454.0,6.714171,0,4,...,6.900731,0.190701,6.419995,-0.317321,6.619884,-0.031093,0.021830,0.029714,0.480736,0.508023
1,浙江,330000,f8a6975573af1b33,Sedan,2016,10,752.0,6.752270,0,3,...,6.752270,0.190701,6.419995,-0.174788,6.598675,0.028144,0.012616,0.013348,0.332275,0.365489
2,浙江,330000,f8a6975573af1b33,Sedan,2016,11,503.0,6.543912,0,2,...,6.752270,0.190701,6.419995,-0.208359,6.593033,-0.016429,0.012979,0.020524,0.332275,0.399060
3,浙江,330000,f8a6975573af1b33,Sedan,2016,12,780.0,7.282074,0,1,...,7.282074,0.738162,6.419995,-0.208359,6.684402,0.135729,0.072106,0.088094,0.862079,0.946520
4,浙江,330000,f8a6975573af1b33,Sedan,2017,1,679.0,6.622736,1,0,...,7.282074,0.738162,6.419995,-0.659337,6.706551,0.033790,0.065451,0.177449,0.862079,1.397499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17155,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,5,991.0,6.322565,0,4,...,7.367709,0.470004,5.768321,-1.131402,6.573917,-0.100995,0.241924,0.324153,1.599388,1.601406
17156,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,6,1189.0,6.580639,0,5,...,7.367709,0.470004,5.768321,-1.131402,6.524217,-0.131178,0.221495,0.296100,1.599388,1.601406
17157,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,7,1206.0,6.577861,0,6,...,6.899723,0.470004,5.768321,-1.131402,6.411382,-0.053644,0.107535,0.273930,1.131402,1.601406
17158,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,8,1267.0,6.898715,0,6,...,6.898715,0.470004,5.768321,-0.169675,6.411238,0.188399,0.107394,0.045126,1.130394,0.639678


     province  adcode             model bodyType  regYear  regMonth  \
0          浙江  330000  f8a6975573af1b33    Sedan     2017        12   
1          福建  350000  f8a6975573af1b33    Sedan     2017        12   
2          四川  510000  f8a6975573af1b33    Sedan     2017        12   
3          陕西  610000  f8a6975573af1b33    Sedan     2017        12   
4          安徽  340000  f8a6975573af1b33    Sedan     2017        12   
...       ...     ...               ...      ...      ...       ...   
1315       重庆  500000  c6cd4e0e073f5ac2    Sedan     2017        12   
1316       河北  130000  c6cd4e0e073f5ac2    Sedan     2017        12   
1317       山西  140000  c6cd4e0e073f5ac2    Sedan     2017        12   
1318       江西  360000  c6cd4e0e073f5ac2    Sedan     2017        12   
1319       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        12   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        7.357556                  0                         2    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
                            ..
salesVolume_his_diff_aver    0
salesVolume_his_var          0
salesVolume_his_diff_var     0
salesVolume_his_pth          0
salesVolume_his_diff_pth     0
Length: 68, dtype: int64

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


In [13]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2020,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.67661
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.63608
[3]	valid_0's rmse: 1.60419
[4]	valid_0's rmse: 1.56554
[5]	valid_0's rmse: 1.53001
[6]	valid_0's rmse: 1.49493
[7]	valid_0's rmse: 1.46117
[8]	valid_0's rmse: 1.43341
[9]	valid_0's rmse: 1.40048
[10]	valid_0's rmse: 1.36974
[11]	valid_0's rmse: 1.33973
[12]	valid_0's rmse: 1.31116
[13]	valid_0's rmse: 1.28416
[14]	valid_0's rmse: 1.2586
[15]	valid_0's rmse: 1.23258
[16]	valid_0's rmse: 1.20933
[17]	valid_0's rmse: 1.18622
[18]	valid_0's rmse: 1.16473
[19]	valid_0's rmse: 1.1445
[20]	valid_0's rmse: 1.12326
[21]	valid_0's rmse: 1.10363
[22]	valid_0's rmse: 1.08429
[23]	valid_0's rmse: 1.06519
[24]	valid_0's rmse: 1.04732
[25]	valid_0's rmse: 1.02768
[26]	valid_0's rmse: 1.0097
[27]	valid_0's rmse: 0.99291
[28]	valid_0's rmse: 0.982142
[29]	valid_0's rmse: 0.965968
[30]	valid_0's rmse: 0.951423
[31]	valid_0's rmse: 0.937206
[32]	valid_0's rmse: 0.924204
[33]	valid_0's r

[898.57322441 432.82226938 549.25372204 ... 111.58283083 146.83960416
 485.03260715]
model train over, rmse: 0.5978106478325848


```
model train over, rmse: 0.6039092447697354
```

### Model-LightGBM - 四月

In [14]:
size, pre = 4, 7   # 5
train_feature  = get_train_feature(size, pre)
test_feature  = get_test_feature(size, pre)

cols = ['province', 'adcode', 'model', 'regYear', 'regMonth', 'bodyType']   #  , 'salesVolume'

temp_train = get_basic_feature(size, pre, 'train')
train_feature = train_feature.drop(drop_cols, axis=1).merge(temp_train, on=cols, how='left')
train_feature

temp_test = get_basic_feature(size, pre, 'test')
test_feature = test_feature.drop(drop_cols, axis=1).merge(temp_test, on=cols, how='left')
test_feature
train_feature.isnull().sum()
test_feature

submit = test_feature[['province', 'adcode', 'model']]
submit['regYear'] = 2018
submit['regMonth'] = 4
###############################

test_index = list(train_feature[(train_feature['regYear'] == 2017) & (train_feature['regMonth'] == 8)].index)

def drop_duplicate(n):
    return n not in test_index

train_index = list(filter(drop_duplicate, list(range(len(train_feature)))))

train_model = train_feature['model'].values[train_index]   # model
val_model = train_feature['model'].values[test_index]

model_set = dict()
for index in range(len(cars)):
    model_set[cars[index]] = index
train_feature['bodyType'] = train_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
train_feature['model'] = train_feature['model'].map(model_set)
test_feature['bodyType'] = test_feature['bodyType'].map({'Hatchback': 0, 'MPV': 1, 'SUV': 2, 'Sedan': 3})
test_feature['model'] = test_feature['model'].map(model_set)

train_label = train_feature[['label']]
train_feature.drop(['province', 'label'], axis=1, inplace=True)
test_feature.drop(['province', 'label'], axis=1, inplace=True)

train_label['log'] = train_label['label'].apply(lambda index: np.log2(index) + 1)
x_train = train_feature.values[train_index]
y_train = train_label['log'].values[train_index]
x_test = train_feature.values[test_index]
y_test = train_label['log'].values[test_index]

      province  adcode             model bodyType  regYear  regMonth  \
0           浙江  330000  f8a6975573af1b33    Sedan     2016         8   
1           浙江  330000  f8a6975573af1b33    Sedan     2016         9   
2           浙江  330000  f8a6975573af1b33    Sedan     2016        10   
3           浙江  330000  f8a6975573af1b33    Sedan     2016        11   
4           浙江  330000  f8a6975573af1b33    Sedan     2016        12   
...        ...     ...               ...      ...      ...       ...   
17155       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         4   
17156       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         5   
17157       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         6   
17158       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         7   
17159       河南  410000  c6cd4e0e073f5ac2    Sedan     2017         8   

       salesVolume  is_pring_festival  distance_spring_festival  \
0         6.610696                  0                         5   
1

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2016,8,1454.0,6.610696,0,5,...,6.900731,0.377168,6.419995,-0.317321,6.592654,0.014522,0.021143,0.052395,0.480736,0.694490
1,浙江,330000,f8a6975573af1b33,Sedan,2016,9,752.0,6.714171,0,4,...,6.900731,0.190701,6.419995,-0.317321,6.619884,-0.031093,0.021830,0.029714,0.480736,0.508023
2,浙江,330000,f8a6975573af1b33,Sedan,2016,10,503.0,6.752270,0,3,...,6.752270,0.190701,6.419995,-0.174788,6.598675,0.028144,0.012616,0.013348,0.332275,0.365489
3,浙江,330000,f8a6975573af1b33,Sedan,2016,11,780.0,6.543912,0,2,...,6.752270,0.190701,6.419995,-0.208359,6.593033,-0.016429,0.012979,0.020524,0.332275,0.399060
4,浙江,330000,f8a6975573af1b33,Sedan,2016,12,679.0,7.282074,0,1,...,7.282074,0.738162,6.419995,-0.208359,6.684402,0.135729,0.072106,0.088094,0.862079,0.946520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17155,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,4,991.0,6.492240,0,3,...,7.367709,0.470004,5.768321,-1.131402,6.651535,-0.062275,0.239052,0.326331,1.599388,1.601406
17156,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,5,1189.0,6.322565,0,4,...,7.367709,0.470004,5.768321,-1.131402,6.573917,-0.100995,0.241924,0.324153,1.599388,1.601406
17157,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,6,1206.0,6.580639,0,5,...,7.367709,0.470004,5.768321,-1.131402,6.524217,-0.131178,0.221495,0.296100,1.599388,1.601406
17158,河南,410000,c6cd4e0e073f5ac2,Sedan,2017,7,1267.0,6.577861,0,6,...,6.899723,0.470004,5.768321,-1.131402,6.411382,-0.053644,0.107535,0.273930,1.131402,1.601406


     province  adcode             model bodyType  regYear  regMonth  \
0          浙江  330000  f8a6975573af1b33    Sedan     2017        12   
1          福建  350000  f8a6975573af1b33    Sedan     2017        12   
2          四川  510000  f8a6975573af1b33    Sedan     2017        12   
3          陕西  610000  f8a6975573af1b33    Sedan     2017        12   
4          安徽  340000  f8a6975573af1b33    Sedan     2017        12   
...       ...     ...               ...      ...      ...       ...   
1315       重庆  500000  c6cd4e0e073f5ac2    Sedan     2017        12   
1316       河北  130000  c6cd4e0e073f5ac2    Sedan     2017        12   
1317       山西  140000  c6cd4e0e073f5ac2    Sedan     2017        12   
1318       江西  360000  c6cd4e0e073f5ac2    Sedan     2017        12   
1319       河南  410000  c6cd4e0e073f5ac2    Sedan     2017        12   

      salesVolume  is_pring_festival  distance_spring_festival  salesVolume_1  \
0        7.357556                  0                         2    

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


province                     0
adcode                       0
model                        0
bodyType                     0
regYear                      0
                            ..
salesVolume_his_diff_aver    0
salesVolume_his_var          0
salesVolume_his_diff_var     0
salesVolume_his_pth          0
salesVolume_his_diff_pth     0
Length: 61, dtype: int64

,province,adcode,model,bodyType,regYear,regMonth,label,salesVolume,is_pring_festival,distance_spring_festival,...,salesVolume_his_max,salesVolume_his_diff_max,salesVolume_his_min,salesVolume_his_diff_min,salesVolume_his_aver,salesVolume_his_diff_aver,salesVolume_his_var,salesVolume_his_diff_var,salesVolume_his_pth,salesVolume_his_diff_pth
0,浙江,330000,f8a6975573af1b33,Sedan,2017,12,NaN,7.357556,0,2,...,7.357556,0.363114,6.632002,-0.125718,7.001934,0.120926,0.053544,0.026056,0.725554,0.488831
1,福建,350000,f8a6975573af1b33,Sedan,2017,12,NaN,6.605298,0,2,...,6.605298,0.190729,6.006353,-0.220184,6.296365,0.063127,0.032470,0.021035,0.598945,0.410912
2,四川,510000,f8a6975573af1b33,Sedan,2017,12,NaN,6.864848,0,2,...,6.864848,0.329607,6.165418,-0.247408,6.437167,0.109376,0.051337,0.038224,0.699430,0.577015
3,陕西,610000,f8a6975573af1b33,Sedan,2017,12,NaN,5.480639,0,2,...,5.480639,0.293253,5.105945,-0.135624,5.264980,0.062449,0.018698,0.024172,0.374693,0.428877
4,安徽,340000,f8a6975573af1b33,Sedan,2017,12,NaN,6.834109,0,2,...,6.834109,0.340355,6.212606,-0.120674,6.451389,0.083472,0.033873,0.021090,0.621503,0.461028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,重庆,500000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,4.812184,0,2,...,4.812184,0.356675,3.828641,-0.287682,4.287344,0.080242,0.095821,0.061696,0.983543,0.644357
1316,河北,130000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.361302,0,2,...,6.361302,0.367341,5.652489,-0.143569,5.949899,0.094207,0.041561,0.034474,0.708813,0.510910
1317,山西,140000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,5.645447,0,2,...,6.324359,1.287406,5.036953,-0.647605,5.537708,0.101416,0.148402,0.379318,1.287406,1.935012
1318,江西,360000,c6cd4e0e073f5ac2,Sedan,2017,12,NaN,6.327937,0,2,...,6.327937,0.341258,5.521461,0.021931,5.950425,0.134413,0.074671,0.011275,0.806476,0.319328


In [15]:
# LightGBM model
params = {
      'boosting_type': 'gbdt',
      'objective': 'rmse',
      'metric': ['rmse'],   # 'l2', 'binary_logloss',
      'learning_rate': 0.03,
      'num_leaves': 2 ** 5 - 1,
      # 'min_child_samples': 100,
      'max_depth': 6,
      'subsample': 0.8,
      'subsample_freq': 5,
      'colsample_bytree': 0.8,
      'seed': 2020,
      'nthread': -1,
      'verbose': 1,
}

lgb_train = lgb.Dataset(x_train, y_train.ravel())
lgb_eval = lgb.Dataset(x_test, y_test.ravel(), reference=lgb_train)
# categorial_name = ['adcode', 'model', 'bodyType', 'regYear', 'regMonth']

module = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=lgb_eval, early_stopping_rounds=100, categorical_feature=categorial_name)

val = module.predict(x_test, num_iteration=module.best_iteration)
val = 2 ** (val - 1)
y_true = 2 ** (y_test.reshape(1, -1)[0] - 1)
nrmse = metrics(y_true, val, val_model.reshape(1, -1)[0])

iters = module.best_iteration + 100
train_all = np.vstack((x_train, x_test))
label_all = np.hstack((y_train, y_test))
lgb_data = lgb.Dataset(train_all, label_all.ravel())
model = lgb.train(params, lgb_data, num_boost_round=iters, categorical_feature=categorial_name)

predict = model.predict(test_feature)
predict = 2 ** (predict - 1)
print(predict)

print('model train over, rmse:', nrmse)
submit['forecastVolum'] = predict
test_prob_collection = pd.concat([test_prob_collection, submit], axis=0, ignore_index=True)

[1]	valid_0's rmse: 1.67973
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 1.64004
[3]	valid_0's rmse: 1.60116
[4]	valid_0's rmse: 1.56669
[5]	valid_0's rmse: 1.5308
[6]	valid_0's rmse: 1.49493
[7]	valid_0's rmse: 1.46497
[8]	valid_0's rmse: 1.43153
[9]	valid_0's rmse: 1.40306
[10]	valid_0's rmse: 1.3705
[11]	valid_0's rmse: 1.34095
[12]	valid_0's rmse: 1.31212
[13]	valid_0's rmse: 1.28737
[14]	valid_0's rmse: 1.26117
[15]	valid_0's rmse: 1.23771
[16]	valid_0's rmse: 1.21383
[17]	valid_0's rmse: 1.19183
[18]	valid_0's rmse: 1.17006
[19]	valid_0's rmse: 1.14914
[20]	valid_0's rmse: 1.12797
[21]	valid_0's rmse: 1.10773
[22]	valid_0's rmse: 1.08823
[23]	valid_0's rmse: 1.07107
[24]	valid_0's rmse: 1.05773
[25]	valid_0's rmse: 1.04237
[26]	valid_0's rmse: 1.02648
[27]	valid_0's rmse: 1.01175
[28]	valid_0's rmse: 0.998648
[29]	valid_0's rmse: 0.984618
[30]	valid_0's rmse: 0.973183
[31]	valid_0's rmse: 0.960024
[32]	valid_0's rmse: 0.947877
[33]	valid_0's 

[280]	valid_0's rmse: 0.637809
[281]	valid_0's rmse: 0.637784
[282]	valid_0's rmse: 0.638004
[283]	valid_0's rmse: 0.638074
[284]	valid_0's rmse: 0.638197
[285]	valid_0's rmse: 0.63835
[286]	valid_0's rmse: 0.636955
[287]	valid_0's rmse: 0.636935
[288]	valid_0's rmse: 0.636846
[289]	valid_0's rmse: 0.636424
[290]	valid_0's rmse: 0.636267
[291]	valid_0's rmse: 0.636628
[292]	valid_0's rmse: 0.636812
[293]	valid_0's rmse: 0.636942
[294]	valid_0's rmse: 0.63725
[295]	valid_0's rmse: 0.637344
[296]	valid_0's rmse: 0.637518
[297]	valid_0's rmse: 0.637141
[298]	valid_0's rmse: 0.637164
[299]	valid_0's rmse: 0.637563
[300]	valid_0's rmse: 0.637652
[301]	valid_0's rmse: 0.637634
[302]	valid_0's rmse: 0.637491
[303]	valid_0's rmse: 0.637401
[304]	valid_0's rmse: 0.637421
[305]	valid_0's rmse: 0.637368
[306]	valid_0's rmse: 0.637315
[307]	valid_0's rmse: 0.63734
[308]	valid_0's rmse: 0.637389
[309]	valid_0's rmse: 0.637362
[310]	valid_0's rmse: 0.637074
[311]	valid_0's rmse: 0.636888
[312]	valid

[564]	valid_0's rmse: 0.63536
[565]	valid_0's rmse: 0.635352
[566]	valid_0's rmse: 0.635291
[567]	valid_0's rmse: 0.635366
[568]	valid_0's rmse: 0.635541
[569]	valid_0's rmse: 0.635552
[570]	valid_0's rmse: 0.635661
[571]	valid_0's rmse: 0.635705
[572]	valid_0's rmse: 0.635645
[573]	valid_0's rmse: 0.635594
[574]	valid_0's rmse: 0.635505
Early stopping, best iteration is:
[474]	valid_0's rmse: 0.633342
0.3316704080482528
0.2568025226371792
0.41891640981415323
0.09862534434881365
0.3749036515320365
1.0784307265452855
0.1786504669472129
0.1890259656433429
0.3617409424317229
0.15471318904954245
0.5341985116231796
0.19886703890955662
0.15946327900008986
0.5379378013805817
0.36201729196960025
0.27797725273290247
0.1240418501097146
0.21696517969277654
0.20729458531302236
0.5252144304959323
0.27195199400639203
0.27774656674449294
0.1666924246269019
0.32464176169237274
0.3750744022885043
0.4951248481741097
0.2518198829675532
0.17720362160304495
0.22608656592364906
0.39232688337559096
0.1841100

```
model train over, rmse: 0.5910535328933109
```

In [16]:
train_feature.shape

(17160, 59)

In [17]:
test_prob_collection

,province,adcode,model,regYear,regMonth,forecastVolum
0,浙江,330000,f8a6975573af1b33,2018,1,856.817857
1,福建,350000,f8a6975573af1b33,2018,1,445.930417
2,四川,510000,f8a6975573af1b33,2018,1,657.344499
3,陕西,610000,f8a6975573af1b33,2018,1,161.944001
4,安徽,340000,f8a6975573af1b33,2018,1,640.131221
...,...,...,...,...,...,...
5275,重庆,500000,c6cd4e0e073f5ac2,2018,4,33.687490
5276,河北,130000,c6cd4e0e073f5ac2,2018,4,157.531719
5277,山西,140000,c6cd4e0e073f5ac2,2018,4,94.524468
5278,江西,360000,c6cd4e0e073f5ac2,2018,4,138.585341


In [18]:
test_prob_collection.index = range(len(test_prob_collection))
evaluation_public = evaluation_public.merge(test_prob_collection, on=['province', 'adcode', 'model', 'regYear', 'regMonth'], how='left')
evaluation_public['forecastVolum'] = evaluation_public['forecastVolum_y']
evaluation_public['forecastVolum'] = evaluation_public['forecastVolum'].apply(lambda index: int(np.round(index)))
evaluation_public['forecastVolum'].mean()

472.8589015151515

In [19]:
evaluation_public[['id', 'forecastVolum']].to_csv('./submit/sub_method_one.csv', encoding='utf-8', index=None)

In [20]:
evaluation_public.describe()
evaluation_public.groupby(['regMonth'], as_index=False)['forecastVolum'].mean()

,id,adcode,regYear,regMonth,forecastVolum_x,forecastVolum_y,forecastVolum
count,5280.000000,5280.000000,5280.0,5280.00000,0.0,5280.000000,5280.000000
mean,2677.166667,347727.272727,2018.0,2.50000,NaN,472.854266,472.858902
std,1549.503211,136292.426601,0.0,1.11814,NaN,586.277778,586.280219
min,1.000000,110000.000000,2018.0,1.00000,NaN,7.264274,7.000000
25%,1342.750000,230000.000000,2018.0,1.75000,NaN,128.738503,129.000000
50%,2684.500000,355000.000000,2018.0,2.50000,NaN,282.989096,283.000000
75%,4026.250000,440000.000000,2018.0,3.25000,NaN,570.220219,570.000000
max,5368.000000,610000.000000,2018.0,4.00000,NaN,6880.702220,6881.000000


,regMonth,forecastVolum
0,1,559.806061
1,2,374.103788
2,3,474.549242
3,4,482.976515
